In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os
import re
import math

In [57]:
df = pd.read_csv(r'C:\Facultate\Tensor-Reloaded\Cascading-Sum-Augmentation\results_cifar=subset_lvl=model.csv', header=None)
df.columns = ['Model', 'Attack', 'CleanAcc', 'AdvAcc']

In [58]:
df

,Model,Attack,CleanAcc,AdvAcc
0,06-18_17-52-33_baseline_subset=1000,fgsm,0.4356,0.0051
1,06-20_02-14-56_4_lips=True_llvl=model_llf=0.01...,fgsm,0.4284,0.0060
2,06-20_02-14-56_5_lips=True_llvl=model_llf=0.1_...,fgsm,0.4290,0.0078
3,06-20_15-21-28_0_lips=True_llvl=model_llf=0.00...,fgsm,0.4410,0.0047
4,06-20_15-21-28_1_lips=True_llvl=model_llf=0.05...,fgsm,0.4382,0.0130
...,...,...,...,...
59,06-18_22-36-17_0_homo=True_hf=0.5_hlvl=model_s...,pgd,0.3734,0.0085
60,06-22_11-49-06_0_lips=True_homo=True_hf=0.05_h...,pgd,0.5125,0.0007
61,06-22_11-49-06_1_lips=True_homo=True_hf=0.1_hl...,pgd,0.4995,0.0010
62,06-22_11-49-06_2_lips=True_homo=True_hf=0.05_h...,pgd,0.5132,0.0008


In [77]:
from collections import defaultdict

def generate_pretty_model_name(s):
    if 'baseline' in s:
        return 'Baseline'
    
    homo = 'homo=True' in s
    lips = 'lips=True' in s
    
    hf = lambda s: re.findall('hf=[^_]+', s)[0][3:]
    llf = lambda s: re.findall('llf=[^_]+', s)[0][4:]
    lnf = lambda s: re.findall('lnf=[^_]+', s)[0][4:]
    
    parts = []
    
    if 'homo=True' in s:
        parts.append('HFact=' + hf(s))
    
    if 'lips=True' in s:
        parts.append('LFact=' + llf(s))
        parts.append('LNoiseFact=' + lnf(s))
    
    return '_'.join(parts)
    

def transform(df):
    models = df['Model'].unique()
    attacks = df['Attack'].unique()
    
    data = defaultdict(dict)
    
    for model in models:
        for attack in attacks:
            row = df.loc[(df['Model'] == model) & (df['Attack'] == attack)]
            data[model][attack.upper()] = (row['CleanAcc'].item(), row['AdvAcc'].item())
    
    final = defaultdict(dict)
    for model, v in data.items():
        clean_accs = [vv[0] for vv in v.values()]
        assert all(a == clean_accs[0] for a in clean_accs)
        
        final[model]['Clean'] = clean_accs[0]
        for attack, vv in v.items():
            final[model][attack] = vv[1]
        
    final = {
        generate_pretty_model_name(model): v for model, v in final.items()
    }
    
    final = {
        model: {
            'Model': model,
            **{
                attack: f'{v * 100:.3f}%' 
                for attack, v in vals.items()
            },
        }
        for model, vals in final.items()
    }
    
    return pd.DataFrame.from_dict(final, orient='index').reset_index(drop=True)
            
pretty_df = transform(df)

In [94]:
pretty_df

,Model,Clean,FGSM,BIM,MIM,PGD
0,Baseline,43.560%,0.510%,0.040%,0.080%,0.020%
1,LFact=0.01_LNoiseFact=0.35,42.840%,0.600%,0.070%,0.100%,0.040%
2,LFact=0.1_LNoiseFact=0.35,42.900%,0.780%,0.200%,0.300%,0.070%
3,LFact=0.005_LNoiseFact=0.2,44.100%,0.470%,0.060%,0.140%,0.020%
4,LFact=0.05_LNoiseFact=0.2,43.820%,1.300%,0.300%,0.440%,0.190%
5,HFact=0.1,51.170%,2.440%,0.140%,0.190%,0.040%
6,HFact=0.5,37.340%,4.610%,1.380%,1.750%,0.850%
7,HFact=4.0,49.560%,2.090%,0.040%,0.110%,0.000%
8,HFact=10.0,41.230%,3.580%,0.800%,1.020%,0.490%
9,HFact=0.01,47.120%,0.560%,0.020%,0.030%,0.010%


In [97]:
def key(row):
    model = row[0]
    if 'Baseline' in model:
        return [0, 0, 0, 0]
    ret = [0, 0, 0, 0]
    
    get = lambda s, substr: float(re.findall(f'{substr}=[^_]+', s)[0].split('=')[1])
    if 'HFact' in model:
        ret[0] += 1
        ret[2] += get(model, 'HFact')
        
    if 'LFact' in model:
        ret[0] += 1
        ret[1] += 1
        ret[3] += get(model, 'LFact')
    return ret

sorted_df = pd.DataFrame(sorted(pretty_df.values.tolist(), key=key), columns=pretty_df.columns)

In [98]:
print(sorted_df.to_latex(index=False))

\begin{tabular}{llllll}
\toprule
                                  Model &    Clean &    FGSM &     BIM &     MIM &     PGD \\
\midrule
                               Baseline &  43.560\% &  0.510\% &  0.040\% &  0.080\% &  0.020\% \\
                             HFact=0.01 &  47.120\% &  0.560\% &  0.020\% &  0.030\% &  0.010\% \\
                              HFact=0.1 &  51.170\% &  2.440\% &  0.140\% &  0.190\% &  0.040\% \\
                              HFact=0.5 &  37.340\% &  4.610\% &  1.380\% &  1.750\% &  0.850\% \\
                              HFact=4.0 &  49.560\% &  2.090\% &  0.040\% &  0.110\% &  0.000\% \\
                             HFact=10.0 &  41.230\% &  3.580\% &  0.800\% &  1.020\% &  0.490\% \\
             LFact=0.005\_LNoiseFact=0.2 &  44.100\% &  0.470\% &  0.060\% &  0.140\% &  0.020\% \\
             LFact=0.01\_LNoiseFact=0.35 &  42.840\% &  0.600\% &  0.070\% &  0.100\% &  0.040\% \\
              LFact=0.05\_LNoiseFact=0.2 &  43.820\% &  1.300\% &  0.3